# 🧠 O-ISAC CoT Master Pipeline (V4)

**"Optical Integrated Sensing and Communication"** Sistematik Derlemesi için Ana Yönetim Paneli.

**Aşamalar:**
1. 📦 Setup & Mount
2. 🏭 Phase 1: Data Prep (PDF → Markdown)
3. 👁️ Phase 2: Visual Analysis (Gemini Vision)
4. 🧠 Phase 3: Integrated Reasoning (V4 Llama Engine) **[NEW]**
5. 📊 Results & Export

**Gereksinimler:**
- Colab GPU Runtime (T4 veya A100)
- GROQ_API_KEY (Colab Secrets)
- GOOGLE_API_KEY (Colab Secrets)

---
**Son Güncelleme:** 2025-12-13
**Versiyon:** 4.0 (The Factory)

---
## 📦 Section 1: Setup & Mount

In [5]:
# @title 1.1 Install Dependencies
# Phase 1 & 2 dependencies
!pip install marker-pdf -q
!pip install transformers torch pillow -q

# Phase 3 & V4 Engine dependencies
!pip install groq nest_asyncio pandas pyyaml -q
!pip install -q -U google-generativeai

print("✅ Tüm bağımlılıklar yüklendi!")

✅ Tüm bağımlılıklar yüklendi!


In [6]:
# @title 1.2 Mount Google Drive & Setup Paths
from google.colab import drive
from google.colab import userdata
import os
import sys

# Mount Drive
drive.mount('/content/drive')

# Project Paths
PROJECT_ROOT = '/content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST'
NOTEBOOKS_DIR = os.path.join(PROJECT_ROOT, 'analysis/notebooks')
COT_LAB_DIR = os.path.join(PROJECT_ROOT, 'analysis/cot_laboratory')
PDF_DIR = os.path.join(PROJECT_ROOT, 'data/retrieved_docs')
MARKDOWN_DIR = os.path.join(PROJECT_ROOT, 'data/processed_markdowns')
OUTPUT_DIR_V4 = os.path.join(PROJECT_ROOT, 'data/extraction_results_v4')

# Add to Python Path
sys.path.insert(0, NOTEBOOKS_DIR)
sys.path.insert(0, PROJECT_ROOT)

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"📄 PDF Directory: {PDF_DIR}")
print(f"📝 Markdown Directory: {MARKDOWN_DIR}")
print(f"📊 V4 Output Directory: {OUTPUT_DIR_V4}")
print("✅ Paths configured!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 Project Root: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST
📄 PDF Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/retrieved_docs
📝 Markdown Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/processed_markdowns
📊 V4 Output Directory: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/extraction_results_v4
✅ Paths configured!


In [7]:
# @title 1.3 Load API Keys
try:
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ API Keys (Groq + Google) yüklendi!")
except Exception as e:
    print("❌ HATA: Sol menüden 🔑 Secrets bölümüne API anahtarlarını ekleyin!")
    print(f"   Hata detayı: {e}")

✅ API Keys (Groq + Google) yüklendi!


---
## 🏭 Section 2: Phase 1 - Digitalization (PDF → Markdown)

**⚠️ GPU Gerektirir!** Bu adım PDF'leri OCR ile markdown'a çevirir.
*Engine: `marker-pdf`*

In [8]:
# @title 2.1 Import & Status Check
import extraction_pipeline_v3 as v3
from extraction_pipeline_v3 import Config

# Initialize
Config.init_dirs()
checkpoint = v3.CheckpointManager(Config.CHECKPOINT_FILE)

# Show status
processed_count = len(checkpoint.data.get('processed', {}))
import glob
pdf_count = len(glob.glob(os.path.join(PDF_DIR, '*.pdf')))

print(f"📊 PDF Durumu: {pdf_count} toplam, {processed_count} işlenmiş.")

📊 PDF Durumu: 32 toplam, 32 işlenmiş.


In [9]:
# @title 2.2 Run Digitization (Phase 1)
# ⚠️ paper başına ~2 dk sürer

FORCE_REPROCESS = False # @param {type:"boolean"}

print("⏳ Phase 1: Dijitalleştirme başlıyor...")
v3.phase1_marker_conversion(checkpoint, force_all=FORCE_REPROCESS)
print("✅ Phase 1 tamamlandı!")

⏳ Phase 1: Dijitalleştirme başlıyor...

📄 PHASE 1: PDF → MARKDOWN (Marker)
Found 32 PDFs
   ⏩ O_ISAC_001 - already processed, skipping
   ⏩ O_ISAC_002 - already processed, skipping
   ⏩ O_ISAC_003 - already processed, skipping
   ⏩ O_ISAC_004 - already processed, skipping
   ⏩ O_ISAC_005 - already processed, skipping
   ⏩ O_ISAC_006 - already processed, skipping
   ⏩ O_ISAC_007 - already processed, skipping
   ⏩ O_ISAC_008 - already processed, skipping
   ⏩ O_ISAC_009 - already processed, skipping
   ⏩ O_ISAC_010 - already processed, skipping
   ⏩ O_ISAC_011 - already processed, skipping
   ⏩ O_ISAC_012 - already processed, skipping
   ⏩ O_ISAC_013 - already processed, skipping
   ⏩ O_ISAC_014 - already processed, skipping
   ⏩ O_ISAC_015 - already processed, skipping
   ⏩ O_ISAC_016 - already processed, skipping
   ⏩ O_ISAC_017 - already processed, skipping
   ⏩ O_ISAC_018 - already processed, skipping
   ⏩ O_ISAC_019 - already processed, skipping
   ⏩ O_ISAC_020 - already processed, 

---
## 🖼️ Section 3: Phase 2 - Visual Analysis

Grafikleri ve şemaları anlamlandırır.
*Engine: `Gemini 2.5 Flash`*

In [10]:
# @title 3.1 Run Visual Analysis
print("⏳ Phase 2: Görsel analiz başlıyor...")
v3.phase2_visual_analysis(checkpoint)
print("✅ Phase 2 tamamlandı!")

⏳ Phase 2: Görsel analiz başlıyor...

👁️ PHASE 2: VISUAL ANALYSIS (Gemini 1.5 Flash - BATCHED)
Papers to analyze: 32
   ⏩ O_ISAC_001 - already analyzed, skipping
   ⏩ O_ISAC_002 - already analyzed, skipping
   ⏩ O_ISAC_003 - already analyzed, skipping
   ⏩ O_ISAC_004 - already analyzed, skipping
   ⏩ O_ISAC_005 - already analyzed, skipping
   ⏩ O_ISAC_006 - already analyzed, skipping
   ⏩ O_ISAC_007 - already analyzed, skipping
   ⏩ O_ISAC_008 - already analyzed, skipping
   ⏩ O_ISAC_009 - already analyzed, skipping
   ⏩ O_ISAC_010 - already analyzed, skipping
   ⏩ O_ISAC_011 - already analyzed, skipping
   ⏩ O_ISAC_012 - already analyzed, skipping
   ⏩ O_ISAC_013 - already analyzed, skipping
   ⏩ O_ISAC_014 - already analyzed, skipping
   ⏩ O_ISAC_015 - already analyzed, skipping
   ⏩ O_ISAC_016 - already analyzed, skipping
   ⏩ O_ISAC_017 - already analyzed, skipping
   ⏩ O_ISAC_018 - already analyzed, skipping
   ⏩ O_ISAC_019 - already analyzed, skipping
   ⏩ O_ISAC_020 - already an

---
## 🧠 Section 4: Phase 3 - Integrated Reasoning (V4)

**YENİ:** Hem akıl yürütme (CoT) hem de veri çıkarmayı tek seferde yapar.
*Engine: `Llama 3.3 70B` + `CoTAssembler`*

In [11]:
# @title 4.1 Import V4 Engine
import extraction_pipeline_v4 as v4
from extraction_pipeline_v4 import ConfigV4

# Init V4 environment
ConfigV4.init_dirs()
v4_checkpoint = v4.CheckpointManager(os.path.join(ConfigV4.OUTPUT_DIR, "checkpoint_v4.json"))

print("✅ V4 Engine (Factory) Hazır!")
print(f"📂 V4 Çıktı Hedefi: {ConfigV4.OUTPUT_DIR}")

✅ V4 Engine (Factory) Hazır!
📂 V4 Çıktı Hedefi: /content/drive/MyDrive/AKU_WorkSpace/survey_fdgit/OISAC_PRISMA_COMST/data/extraction_results_v4


In [12]:
# @title 4.2 Run Integrated Extraction (Phase 3)

LIMIT = 5 # @param {type:"integer"}

print(f"🚀 Phase 3: Akıl Yürütme ve Çıkarma (Max {LIMIT} paper)...")

results = v4.phase3_integrated_extraction(v4_checkpoint, limit=LIMIT)

print(f"\n🎉 İşlem Tamamlandı! {len(results)} makale analiz edildi.")

🚀 Phase 3: Akıl Yürütme ve Çıkarma (Max 5 paper)...

🧠 PHASE 3: INTEGRATED REASONING EXTRACTION (CoTAssembler)
Papers to process: 5
[1/5] 🔨 Processing: O_ISAC_001
[INFO] Loading Recipe: analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml...
[INFO] Assembling System Prompt from Modules...
[INFO] Calling Groq API (Model: llama-3.3-70b-versatile)...

[DEBUG] RAW RESPONSE LEN: 7414
[DEBUG] RAW RESPONSE START: {
  "reasoning_trace": [
       {
           "key": "step_0_visual_inspection",
           "type": "string",
           "required": true,
           "description": "MANDATORY: You MUST describe what y...
[INFO] Logging Run Evidence...
[OK] Run Logged: 20251213_121550_O_ISAC_001_llama-3.3-70b-versatile
   ✅ Success
[2/5] 🔨 Processing: O_ISAC_002
[INFO] Loading Recipe: analysis/cot_laboratory/recipes/experiment_v1_full_analysis.yaml...
[INFO] Assembling System Prompt from Modules...
[INFO] Calling Groq API (Model: llama-3.3-70b-versatile)...

[DEBUG] RAW RESPONSE LEN: 8391


---
## 📊 Section 5: Results & Dashboard

In [13]:
# @title 5.1 Show Latest Extractions
import pandas as pd
import glob

csv_path = os.path.join(ConfigV4.OUTPUT_DIR, "extraction_v4_summary.csv")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"📊 Toplam {len(df)} kayıt bulundu.")
    display(df.head())
else:
    print("ℹ️ Henüz sonuç csv dosyası oluşmamış.")

📊 Toplam 5 kayıt bulundu.


,Paper_ID,step_0_visual_inspection,step_1_concept_analysis,step_2_benchmark_verification,step_3_strategic_critique
0,O_ISAC_001,The provided figures show the performance eval...,"The system mechanism is based on CE-OFDM, whic...","The reported metrics, such as EVM and PSD, are...",The solved problems include the mitigation of ...
1,O_ISAC_002,The provided images include diagrams of differ...,The system mechanism involves photonic teraher...,The reported metrics include a data transmissi...,The solved problems include the development of...
2,O_ISAC_003,The paper includes several diagrams and charts...,The system mechanism involves the use of visib...,The paper reports on the simulation results of...,The paper solves the problem of characterizing...
3,O_ISAC_004,"The paper includes several figures, including ...",The proposed system integrates optical fiber s...,The paper reports a salinity sensitivity of 0....,The proposed system solves the problem of inte...
4,O_ISAC_005,"The paper includes several figures and charts,...",The system mechanism involves a UAV-aided mixe...,"The paper reports various metrics, including t...",The paper solves the problem of optimizing the...
